In [1]:
import pandas as pd
pd.options.display.max_columns = None

data = pd.read_csv('data/train.csv')

In [4]:
data.shape

(4167, 74)

Comenzaremos eliminando las columnas que decidimos descartar de mara directa en la exploración inicial.

In [6]:
columns = ['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'listing_url', 'scrape_id', 'last_scraped',
           'picture_url', 'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'calendar_last_scraped', 'license']

data.drop(columns, axis=1, inplace=True)

In [7]:
data.shape

(4167, 61)

Cuanto mayor sea el número de columnas, más complicado será trabajar con ellas. Por eso lo que haremos será reducirlas mucho (aun sabiendo que algunas de las que descartamos pueden aportar valor a nuestro modelo) y a partir de ahí trataremos de afinar el modelo incluyendo algunas de esas columnas eliminadas (si lo considerásemos oportuno).

Empezamos dropeando las columnas que sospechamos que no aportan valor (sin entrar en profundidad en los datos, ya sea por la información que aportan los datos o por el formato en el que se nos proporcionan). También haremos algunas comprobacionas con las que tengamos dudas más específicas.

In [43]:
columns = ['name', 'description', 'neighborhood_overview', 'host_name', 'host_location', 'host_about', 'host_listings_count',
          'neighbourhood', 'property_type', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
          'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
          'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
          'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_accuracy', 'review_scores_cleanliness',
          'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
          'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
          'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']

cleaning = data.drop(columns, axis=1)

In [46]:
cleaning.shape

(4167, 28)

In [45]:
cleaning.columns

Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'room_type',
       'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities',
       'price', 'minimum_nights', 'number_of_reviews', 'first_review',
       'last_review', 'review_scores_rating', 'instant_bookable',
       'reviews_per_month'],
      dtype='object')

Una vez hecha la limpieza general, vamos fila por fila para transformar los datos según consideremos oportuno.

**'host_since'**: los valores son fechas en tipo string. Lo que haremos será establecer como valor el año (en formato int) en lugar de la fecha completa. Esto lo hacemos en base a la creencia de que el año de registro del host influye en el precio esablecido.

In [52]:
cleaning.host_since[0]

'2012-07-24'

In [82]:
cleaning.host_since = cleaning.host_since.apply(lambda x: int(x[:4]))

In [80]:
cleaning.host_since.value_counts()

2014-04-22    19
2018-03-13    15
2021-06-08    15
2018-07-20    14
2016-04-13    14
              ..
2012-05-24     1
2017-02-28     1
2019-05-02     1
2019-01-29     1
2018-03-23     1
Name: host_since, Length: 2050, dtype: int64

**'host_response_time'**: tiene cuatro valores únicos que indican la rapidez de respuesta del host. Podemos entenderlo como una escala ordinal por lo que lo trataremos como tal. Es cierto que este orden no debería ser homogéneo, es decir, no es lo mismo el salto de una hora a unas pocas horas que de un día a varios días o más. No tenemos una base real para esto, por lo que simplificaremos bastante esta distinción. Lo que haremos será "penalizar" a aquellos hosts que responden en varios días o más (entendemos que es el tiempo de respuesta al que el usuario es más sensible).

In [61]:
cleaning.host_response_time.value_counts()

within an hour        1550
within a few hours     573
within a day           556
a few days or more      75
Name: host_response_time, dtype: int64

In [62]:
cleaning.host_response_time = cleaning.host_response_time.replace(['within an hour', 'within a few hours', 'within a day', 'a few days or more'],
                                    [1, 2, 3, 5])

0       2.0
1       NaN
2       1.0
3       1.0
4       2.0
       ... 
4162    NaN
4163    1.0
4164    1.0
4165    1.0
4166    NaN
Name: host_response_time, Length: 4167, dtype: float64

Una vez hecha la transformación nos damos cuenta de que la cantidad de NaN es excesiva y no tendría mucho sentido definir valores sin saber cuál es realmente el tiempo de respuesta (al menos, según nuestro planteamiento). Entedemos que la presencia de NaN no es algo que dependa del host o de la propia plataforma Airbnb, sino que se debe a fallos en la extracción de los datos, lo cual complica aún más el manejo de los valores nulos. Decidimos eliminar la columna 'host_response_time'.

In [71]:
cleaning.host_response_time.isnull().sum()

1413

In [72]:
cleaning.drop('host_response_time', axis=1, inplace=True)

**'host_response_rate'**: siguiendo el planteamiento anterior (gran proporción de NaN y dificultad para gestionarlos), decidimos eliminar esta columna.

In [73]:
cleaning.host_response_rate.isnull().sum()

1413

In [75]:
cleaning.drop('host_response_rate', axis=1, inplace=True)

**'host_acceptance_rate'**: en la linea de lo anterior y teniendo en cuenta que ya dudábamos de la aportación de valor de esta columna, también decidimos eliminar esta columna.

In [77]:
cleaning.host_acceptance_rate.isnull().sum()

1092

In [78]:
cleaning.drop('host_acceptance_rate', axis=1, inplace=True)

**'host_is_superhost'**: los valores son 'f' (false) o 't' (true) en formato string. No hay NaN ni ningún otro valor. Simplemente lo que haremos será sustituir 'f' por 0 y 't' por 1 (en formato int lógicamente).

In [87]:
cleaning.host_is_superhost.value_counts()

f    3081
t    1086
Name: host_is_superhost, dtype: int64

In [93]:
cleaning.host_is_superhost = cleaning.host_is_superhost.replace(['f', 't'], [0, 1])

In [94]:
cleaning.host_is_superhost.value_counts()

0    3081
1    1086
Name: host_is_superhost, dtype: int64

**'host_neighbourhood'**: nos damos cueta de que ya existe otra columna que hace referencia al barrio ('neighbourhood_cleansed') y además esta otra columna tiene un formato más amigable (22 valores únicos). Decidimos eliminar esta columna.

In [97]:
len(cleaning.host_neighbourhood.unique())

52

In [98]:
len(cleaning.neighbourhood_cleansed.unique())

22

In [99]:
cleaning.drop('host_neighbourhood', axis=1, inplace=True)

**'host_total_listings_count'**: nos indica la cantidad de propiedades listadas por el host. Entendemos que esto puede influir en el precio ya que, a mayor número de propiedades, más competitivo podrá ser el precio (especialmente si se trata de inmobiliarias o similares). No contiene valores nulos, por lo que mantenemos la columna como está.

In [103]:
cleaning.host_total_listings_count.isnull().sum()

0

In [100]:
cleaning.head()

,id,host_since,host_is_superhost,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,reviews_per_month
0,0,2012,0,13,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,Noord-Oost,52.39508,4.99186,Private room,3,1.5 shared baths,1.0,NaN,"[""First aid kit"", ""Free parking on premises"", ...",87.0,2,81,2014-08-03,2019-06-15,4.62,f,0.91
1,1,2015,0,0,"['email', 'phone', 'reviews']",t,f,Centrum-Oost,52.36371,4.90745,Entire home/apt,4,1 bath,2.0,2.0,"[""Hot water kettle"", ""Pocket wifi"", ""Stove"", ""...",250.0,2,9,2020-01-26,2021-09-05,5.00,f,0.40
2,2,2018,1,20,"['email', 'phone']",t,t,Zuid,52.35482,4.85329,Hotel room,2,1 private bath,1.0,1.0,"[""Air conditioning"", ""First aid kit"", ""Hangers...",151.0,1,47,2018-11-09,2020-03-07,4.79,t,1.25
3,3,2016,0,1,"['email', 'phone', 'facebook']",t,f,De Baarsjes - Oud-West,52.36446,4.85972,Entire home/apt,2,1.5 baths,1.0,1.0,"[""Hair dryer"", ""Stove"", ""Dishes and silverware...",139.0,2,10,2019-04-22,2019-11-10,5.00,f,0.31
4,4,2013,1,1,"['email', 'phone', 'reviews', 'manual_offline'...",t,t,Zuid,52.34176,4.84782,Private room,2,1.5 baths,1.0,2.0,"[""Hair dryer"", ""Dishes and silverware"", ""Ether...",95.0,2,166,2014-04-08,2019-08-26,4.67,f,1.78
